In [1]:
import pandas as pd
import numpy as np
from datetime import timedelta, date
import time

## Check the day type

In [2]:
delta = 0.05

In [31]:
share_price = pd.read_csv('Share_price_data.csv', encoding= 'big5')

In [32]:
share_price['前日收盤價'] = share_price['收盤價'].shift(periods=-1)

In [33]:
share_price['漲跌'] = ''
for index, val in share_price.iterrows():
    diff = (val['收盤價'] - val['前日收盤價']) / val['前日收盤價']
    if abs(diff) > delta:
        if diff > 0:
            share_price.at[index, '漲跌'] = '+'
        else:
            share_price.at[index, '漲跌'] = '-'
    else:
        share_price.at[index, '漲跌'] = '='

民國轉西元

In [34]:
d = share_price['日期']
for i in range(len(d)):
    d.iloc[i] = d.iloc[i].replace(d.iloc[i][0:3], str(int(d.iloc[i][0:3]) + 1911))

share_price['日期'] = d

C:\Users\ray\Anaconda3\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [35]:
share_price.head()

,日期,收盤價,前日收盤價,漲跌
0,2016/01/04,21.55,21.65,=
1,2016/01/05,21.65,20.10,+
2,2016/01/06,20.10,18.90,+
3,2016/01/07,18.90,20.20,-
4,2016/01/08,20.20,19.80,=


## Filter the related words

In [37]:
related_words = ['華通', 'PCB', 'HDI']

In [9]:
def Related_words_filter(related_words, data, target_columns = ["title", "content"]):
    output_id = []
    for index, val in data.iterrows():
        contain_related_words = False
        for key in related_words:
            for target_column in target_columns:
                if val[target_column] is np.nan:
                    continue
                if key in val[target_column]:
                    contain_related_words = True
                    break
            else:
                continue
            break
        if contain_related_words == True:
            output_id.append(index)
        print (index, end='\r')
    output = data[data.index.isin(output_id)]
    return output

In [10]:
bbs = pd.read_csv('bbs.csv', encoding = 'mbcs')
bbs_cleaned = Related_words_filter(related_words, bbs)
bbs_cleaned.shape[0]

808

In [11]:
news = pd.read_csv('news.csv', encoding = 'mbcs')
news_cleaned = Related_words_filter(related_words, news)
news_cleaned.shape[0]

3916

In [12]:
forum = pd.read_csv('forum.csv', encoding = 'mbcs')
forum_cleaned = Related_words_filter(related_words, forum)
forum_cleaned.shape[0]

61

## Mark the news

In [38]:
n_days = 3

In [39]:
def Mark_the_news(share_price, data):
    data["Stock_rises_or_falls"] = "="
    for index, val in data.iterrows():
        news_date = pd.Timestamp(val['post_time'])
        for i, v in share_price.iterrows():
            if (news_date + pd.Timedelta(days=n_days)).day == pd.Timestamp(v["日期"]).day:
                data.at[index, "Stock_rises_or_falls"] = v["漲跌"]
                break
        print(val['post_time'], end='\r')

In [40]:
Mark_the_news(share_price, bbs_cleaned)

<ipython-input-39-a6810ddbca6d>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["Stock_rises_or_falls"] = "="


In [41]:
Mark_the_news(share_price, news_cleaned)

<ipython-input-39-a6810ddbca6d>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["Stock_rises_or_falls"] = "="


In [42]:
Mark_the_news(share_price, forum_cleaned)

<ipython-input-39-a6810ddbca6d>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["Stock_rises_or_falls"] = "="


## Save the marked news

In [ ]:
bbs_cleaned.head()

In [51]:
bbs_cleaned.to_csv('bbs_marked.csv', index = False)

In [ ]:
news_cleaned.head()

In [52]:
news_cleaned.to_csv('news_marked.csv', index = False)

In [ ]:
forum_cleaned.head()

In [53]:
forum_cleaned.to_csv('forum_marked.csv', index = False)